In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [56]:
def scrape_exportacao():

    sub_opc = 1

    # Loop para acessar outras opções de exportação
    while sub_opc <= 4:

      print("Iniciando raspagem...\n")
      resultados = []
      current_year = datetime.datetime.now().year
      sub_opc_str = str(sub_opc)
      base_url = "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={ano}&subopcao=subopt_0" + sub_opc_str +"&opcao=opt_06"


      # está criando um loop que percorre os anos de 1970 até o ano atual
      for ano in range(1970, current_year + 1):
          print(f"Raspando dados do ano {ano}...")

          # faz uma requisição para a URL que tem o ano formatado dentro dela.
          try:
              response = requests.get(base_url.format(ano=ano))
              response.raise_for_status()

          # Captura qualquer erro de rede e imprime uma mensagem de erro:
          except requests.RequestException as e:
              print(f"Erro ao acessar dados de {ano}: {e}")
              continue

          # Faz o parsing do HTML
          soup = BeautifulSoup(response.content, 'html.parser')

          # Tabela com informações de exportação
          table = soup.find('table', {'class': 'tb_base tb_dados'})

          # Tabela com nome das opções de exportação
          encontrar_opcoes = soup.find_all("table", {"class": "tb_base tb_header no_print"})

          # Caso não encontre a tabela retorna a seguinte mensagem:
          if not table:
              print(f"Sem dados disponíveis para o ano {ano}. Pulando...")
              continue

          # Caso não encontre a linha retorna a seguinte mensagem:
          rows = table.find_all('tr')

          if not rows:
              print(f"Nenhuma linha encontrada para o ano {ano}.")
              continue
          # Removendo o ultimo elemento da lista
          rows = rows[:-1]

          # Variavel para as categorias de exportações
          categoria = None

          # Percorrendo uma lista e pulando o primeiro elemento
          for i, row in enumerate(rows):
              if i == 0:
                  continue

              # Procurando linhas na tabela e extraindo os textos
              cols = row.find_all(['th', 'td'])
              dados = [col.get_text(strip=True) for col in cols]

              # Garantindo que a linha tenha 3 colunas
              if len(dados) == 3:
                  pais, quantidade, valor = dados

                  # Substituindo Valores de quantidade
                  if quantidade == "-":
                      quantidade = "0"

                  # Substituindo Valores de valor
                  if valor == "-":
                      valor = "0"

                  # Capturando nome das opções de exportações
                  for opcao in encontrar_opcoes:
                      nome_opcao = opcao.find("button", {"value": "subopt_0" + sub_opc_str}).text.strip()

                  # add valores em uma lista
                  resultados.append({
                      "ano": ano,
                      "categoria": nome_opcao,
                      "pais": pais,
                      "quantidade": quantidade,
                      "valor": valor
                  })
                  print(f"Ano: {ano} | Categoria: {nome_opcao} | País: {pais} | Quantidade: {quantidade} | Valor(US$): {valor}")
      sub_opc = sub_opc + 1
    print(f"Raspagem concluída. Total de registros: {len(resultados)}")
    return resultados

In [57]:
dados = scrape_exportacao()
df = pd.DataFrame(dados)
df.head()

A saída de streaming foi truncada nas últimas 5000 linhas.
Ano: 1986 | Categoria: Suco de uva | País: Dinamarca | Quantidade: 0 | Valor(US$): 0
Ano: 1986 | Categoria: Suco de uva | País: Dominica, Ilha de | Quantidade: 0 | Valor(US$): 0
Ano: 1986 | Categoria: Suco de uva | País: El Salvador | Quantidade: 0 | Valor(US$): 0
Ano: 1986 | Categoria: Suco de uva | País: Emirados Árabes Unidos | Quantidade: 0 | Valor(US$): 0
Ano: 1986 | Categoria: Suco de uva | País: Equador | Quantidade: 0 | Valor(US$): 0
Ano: 1986 | Categoria: Suco de uva | País: Espanha | Quantidade: 0 | Valor(US$): 0
Ano: 1986 | Categoria: Suco de uva | País: Estados Unidos | Quantidade: 2.591.390 | Valor(US$): 2.315.230
Ano: 1986 | Categoria: Suco de uva | País: Falkland (Malvinas) | Quantidade: 0 | Valor(US$): 0
Ano: 1986 | Categoria: Suco de uva | País: Filipinas | Quantidade: 0 | Valor(US$): 0
Ano: 1986 | Categoria: Suco de uva | País: Finlândia | Quantidade: 85.860 | Valor(US$): 78.786
Ano: 1986 | Categoria: Suco de 

,ano,categoria,pais,quantidade,valor
0,1970,Suco de uva,África do Sul,0,0
1,1970,Suco de uva,"Alemanha, República Democrática da",0,0
2,1970,Suco de uva,Angola,0,0
3,1970,Suco de uva,Antígua e Barbuda,0,0
4,1970,Suco de uva,Antilhas Holandesas,0,0
